In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

In [ ]:
# Load datasets
df_earthquake = pd.read_csv('clean_major_earthquakes.csv', parse_dates=['date']) # parse_dates convert the 'date' column from a string(text) format into a datatime format
df_stocks = pd.read_csv('unclean_225_stocks_data.csv', parse_dates=['Date'])
df_companies = pd.read_csv('N225_companies.csv', sep=';') # the file uses semicolons (;) as a delimiter

C:\Users\dongv\AppData\Local\Temp\ipykernel_24832\3635927244.py:3: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stocks = pd.read_csv('clean_225_stocks_data.csv', parse_dates=['Date'])


In [15]:
# Display dataset
print(df_stocks.head())


         Date                Open              Close  Volume  Ticker
0  2000-01-04   96.17342799484351  95.56473541259766  296000  1332.T
1  2000-01-05   99.82558348831859  95.56473541259766  637000  1332.T
2  2000-01-06  101.65168214575645  99.21691131591797  411000  1332.T
3  2000-01-07   98.60819357245632  102.8690414428711  645000  1332.T
4  2000-01-10   102.8690414428711  102.8690414428711       0  1332.T


In [16]:
# Check for missing, duplucate data

# 1. Remove rows where the 'Date' column doesn't match the expected date format
df_stocks = df_stocks[df_stocks['Date'].str.match(r'\d{4}-\d{2}-\d{2}', na=False)]

# 2. Convert the 'Date' column to datetime
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='%Y-%m-%d', errors='coerce')

# 3. Check for rows with invalid dates
invalid_dates = df_stocks[df_stocks['Date'].isna()]
print("Rows with invalid dates:")
print(invalid_dates)

# 4. Drop rows with invalid dates (optional)
df_stocks = df_stocks.dropna(subset=['Date'])

# 5. Check for missing values in each column
missing_values = df_stocks.isnull().sum()
print("\nMissing values in each column:")
print(missing_values)

# 6. Check for missing rows (e.g., gaps in the Date column)
full_date_range = pd.date_range(start=df_stocks['Date'].min(), end=df_stocks['Date'].max(), freq='D')
missing_dates = full_date_range.difference(df_stocks['Date'])
print("\nMissing dates:")
print(missing_dates)

# 7. Check for duplicate rows
duplicate_rows = df_stocks[df_stocks.duplicated()]
print("\nDuplicate rows:")
print(duplicate_rows)

# 8. Check for invalid values (e.g., Volume = 0)
invalid_volume = df_stocks[df_stocks['Volume'] == 0]
print("\nRows with Volume = 0:")
print(invalid_volume)

Rows with invalid dates:
Empty DataFrame
Columns: [Date, Open, Close, Volume, Ticker]
Index: []

Missing values in each column:
Date      0
Open      0
Close     0
Volume    0
Ticker    0
dtype: int64

Missing dates:
DatetimeIndex(['1999-05-08', '1999-05-09', '1999-05-15', '1999-05-16',
               '1999-05-22', '1999-05-23', '1999-05-29', '1999-05-30',
               '1999-06-05', '1999-06-06',
               ...
               '2025-01-19', '2025-01-25', '2025-01-26', '2025-02-01',
               '2025-02-02', '2025-02-08', '2025-02-09', '2025-02-11',
               '2025-02-15', '2025-02-16'],
              dtype='datetime64[ns]', length=2970, freq=None)

Duplicate rows:
Empty DataFrame
Columns: [Date, Open, Close, Volume, Ticker]
Index: []

Rows with Volume = 0:
              Date         Open        Close Volume  Ticker
1310834 2016-07-11  2552.135498  2552.135498      0  9984.T
1311084 2017-07-17  4330.719238  4330.719238      0  9984.T
1311103 2017-08-11  4162.359375  4162.35

In [17]:
# Merge df_stocks with df_companies on 'Ticker'
merged_df = pd.merge(
    df_stocks, 
    df_companies, 
    on='Ticker', 
    how='left'  # Use 'left' to keep all rows from df_stocks
)

# Reorder the columns
column_order = ['Number', 'Ticker', 'Company_name', 'Industry', 'Date', 'Open', 'Close', 'Volume']
merged_df = merged_df[column_order]

# Drop the 'Number' column
merged_df = merged_df.drop(columns=['Number'])

# Filter data to include only rows from the year 2007 onward
merged_df = merged_df[merged_df['Date'] >= '2007-01-01']

 # Convert 'Date' column to datetime format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
    
# Display the resulting DataFrame
print(merged_df.head())

      Ticker  Company_name Industry       Date                Open  \
1824  1332.T  NISSUI CORP.  Fishery 2007-01-04  496.98624286410853   
1825  1332.T  NISSUI CORP.  Fishery 2007-01-05    510.323593674323   
1826  1332.T  NISSUI CORP.  Fishery 2007-01-09   513.8332764679333   
1827  1332.T  NISSUI CORP.  Fishery 2007-01-10  505.40978064903845   
1828  1332.T  NISSUI CORP.  Fishery 2007-01-11   497.6882201100624   

                   Close   Volume  
1824  500.49603271484375   855800  
1825   520.8529663085938  4653800  
1826    510.323486328125  2779700  
1827   492.7745361328125  3638400  
1828  490.66864013671875  2608400  


In [ ]:
# Save result to new csv file
#merged_df.to_csv("clean_225_stocks_data.csv", index=False)